# ANN From Scratch

In [1]:
from graphviz import Digraph
from micrograd.engine import Value

In [2]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
#     assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ %s | data %.4f | grad %.4f }" % (n.label,n.data,n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [3]:
import numpy as np

In [1]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda : None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(label={self.label} data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad 
        out._backward = _backward
        return out
    
    def __radd__(self, other): 
        return self + other
        
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad 
            other.grad += self.data * out.grad 
        out._backward = _backward
        return out
    
    def __rmul__(self, other): 
        return self * other
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data ** other, (self,), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        
        return out
        
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out
    
    def __truediv__(self, other): 
        return self * other**-1
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad 
        out._backward = _backward
        return out

    def __neg__(self): 
        return -self
    
    def __sub__(self, other): 
        return self + (-other)
        
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [9]:
import torch

In [10]:
x1 = torch.Tensor([2.0]).double(); x1.requires_grad = True
x2 = torch.Tensor([0.0]).double(); x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double(); w1.requires_grad = True
w2 = torch.Tensor([1.0]).double(); w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]); b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('-----')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

0.7071066904050358
-----
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w1 1.0000002567688737


In [2]:
import random,math

In [47]:
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.randint(-1,1)) for _ in range(nin)]
        self.b = Value(random.randint(-1,1))
    
    def __call__(self, x):
        act =  sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        return act.tanh()

    def parameters(self):
        return self.w + [self.b]
    
x = [2.0, 3.0]
n = Neuron(2)
n(x)

Value(label= data=0.9999092042625951)

In [48]:
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

In [49]:
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [50]:
mlp = MLP(3,[10,5,2,1])

In [51]:
def train(epochs, model, xs, ygt):
    
    for k in range(epochs):

        ypred = [model(x) for x in xs]
        loss = sum((yout - ygt)**2 for ygt, yout in zip(ygt, ypred))
        
        for p in model.parameters(): p.grad = 0.0 
    
        loss.backward()

        for p in model.parameters(): p.data += -0.01 * p.grad
        if(k%100==0):
            print("Epoch:- ",k," , Loss:- ",loss.data)
    return ypred

In [52]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0], 
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]

In [53]:
y_pred=train(1000, mlp, xs, ys)

Epoch:-  0  , Loss:-  7.94353948634501
Epoch:-  100  , Loss:-  7.8378306175857295
Epoch:-  200  , Loss:-  0.33377841447760764
Epoch:-  300  , Loss:-  0.06232099530939418
Epoch:-  400  , Loss:-  0.03309082314175003
Epoch:-  500  , Loss:-  0.022332407842524996
Epoch:-  600  , Loss:-  0.016793148813542182
Epoch:-  700  , Loss:-  0.013430512461805072
Epoch:-  800  , Loss:-  0.011177332561939353
Epoch:-  900  , Loss:-  0.009564539210702085


In [54]:
print("Predictions :- ",y_pred)

Predictions :-  [Value(label= data=0.9575549071474925), Value(label= data=-0.955993709975816), Value(label= data=-0.9529617052558506), Value(label= data=0.9508666740944983)]


In [55]:
### MLP on iris dataset (Binary Classification)

In [56]:
from sklearn.datasets import load_iris

In [57]:
iris=load_iris()

In [58]:
from sklearn.model_selection import train_test_split

In [66]:
X=iris.data
Y=iris.target
mask = Y < 2  # Include only Setosa (0) and Versicolor (1)
X = X[mask]
Y = Y[mask]

In [67]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,random_state=42,test_size=0.7)

In [68]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((30, 4), (70, 4), (30,), (70,))

In [70]:
mlp = MLP(4,[10,5,1])

In [71]:
train(500,mlp,X_train,y_train)

Epoch:-  0  , Loss:-  10.15912123868205
Epoch:-  100  , Loss:-  0.057909148884709855
Epoch:-  200  , Loss:-  0.020843488582233463
Epoch:-  300  , Loss:-  0.012622345928394212
Epoch:-  400  , Loss:-  0.009056503433395938


[Value(label= data=0.9798558524166964),
 Value(label= data=0.006302477082816038),
 Value(label= data=0.9836427809924796),
 Value(label= data=0.0010658575409127806),
 Value(label= data=0.9787174545399138),
 Value(label= data=0.9825471306284344),
 Value(label= data=0.9833124610137257),
 Value(label= data=-0.009501417535191957),
 Value(label= data=0.983843533027984),
 Value(label= data=0.9838088008011622),
 Value(label= data=0.9839684500628841),
 Value(label= data=0.9839930630716244),
 Value(label= data=-0.010837694164811253),
 Value(label= data=0.013761385632121263),
 Value(label= data=-0.006054873508620611),
 Value(label= data=0.9838425960438782),
 Value(label= data=0.00831867775562339),
 Value(label= data=-0.008549085488479763),
 Value(label= data=0.024520553471045887),
 Value(label= data=0.9839728156301074),
 Value(label= data=0.9838811741710917),
 Value(label= data=0.9832577631119369),
 Value(label= data=0.9837449361375754),
 Value(label= data=0.98307231580837),
 Value(label= data=0.

In [72]:
correct = 0
for x, y in zip(X_test, y_test):
    pred = mlp(x).data
    pred_class = 1 if pred > 0.5 else 0 
    if pred_class == y:
        correct += 1

accuracy = correct / len(y_test)
print(f"Test Accuracy: {accuracy:.2f}")


Test Accuracy: 1.00
